# NULL ANALISYS (ANÁLISIS DE NULOS)

El objetivo de esta sección es analizar las variables con nulos y cómo tratarlas.

<br>

Se encarga de responder preguntas como:
- ¿Qué variables tienen nulos?
- ¿Cuál sería el mejor tratamiento para tales nulos?

<br>

---

## Configuración General

1. Carga de librerías.
2. Seteo de estilos del notebook.
3. Ingesta del dataset.

In [10]:
import sys
import os
import statistics

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt



sys.path.append(os.path.abspath(os.path.join('..', '..', 'src', 'utils')))
import utils as ut

In [11]:
# Seteo de estilos
plt.style.use("ggplot")
sns.set_palette("viridis")
plt.rcParams["figure.figsize"] = (9,6)

In [33]:
wines = pd.read_csv("../../src/data/transformed/wines_transformed.csv")
pd.set_option('display.max_columns', None)
wines.head(3)

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,style,alcohol,image,ageing,black fruit,citrus,dried fruit,earthy,floral,oaky,red fruit,spices,tree fruit,tropical,vegetal,yeasty,any junk food will do,aperitif,appetizers and snacks,beef,blue cheese,cured meat,"game (deer, venison)",goat's milk cheese,lamb,lean fish,mature and hard cheese,mild and soft cheese,mushrooms,pasta,pork,poultry,"rich fish (salmon, tuna etc)",shellfish,spicy food,veal,vegetarian,Albariño,Barbera,Bonarda,Béquignol Noir,Cabernet Franc,Cabernet Sauvignon,Cereza,Chardonnay,Chenin Blanc,Criolla Grande,Garnacha,Gewürztraminer,Grenache,Grüner Veltliner,Malbec,Malvasia,Marsanne,Mencia,Merlot,Moscatel,Mourvedre,Pais,Pedro Ximenez,Petit Verdot,Pinot Gris,Pinot Noir,Riesling,Roussanne,Sangiovese,Sauvignon Blanc,Shiraz/Syrah,Sémillon,Tannat,Tempranillo,Torrontés,Trousseau,Verdejo,Viognier,Agrelo,Argentina,Brazil,Cafayate Valley,Calchaqui Valley,Campanha,Famatina,Gualtallary,La Consulta,La Rioja,Las Compuertas,Lujan de Cuyo,Lunlunta,Maipu,Mendoza,Paraje Altamira,Patagonia,Pedernal Valley,Perdriel,Rio Grande do Sul,Rio Negro,Salta,San Carlos,San Juan,San Rafael,Serra Gaúcha,Tulum Valley,Tunuyán,Tupungato,Uco Valley,Vale dos Vinhedos,Vista Flores
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020.0,Luigi Bosca,4.8,582.0,188.33,0.7343,0.5090,0.1361,0.4474,Argentinian Cabernet Sauvignon - Malbec,NaN,https://images.vivino.com/thumbs/_Bf6JTwYRpSX6...,0.0000,0.3500,0.000,0.000,0.1250,0.0500,0.3250,0.0500,0.1000,0.0,0.0,0.000,0.000,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015.0,Catena Zapata,4.7,297.0,675.00,0.7417,0.5583,0.1434,0.5445,Argentinian Bordeaux Blend,0.14,https://images.vivino.com/thumbs/Yt464jw0QS-ug...,0.0241,0.2008,0.008,0.012,0.0964,0.0241,0.4378,0.0843,0.1004,0.0,0.0,0.004,0.008,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017.0,Catena Zapata,4.7,219.0,580.00,0.7417,0.5583,0.1434,0.5445,Argentinian Bordeaux Blend,NaN,https://images.vivino.com/thumbs/Yt464jw0QS-ug...,0.0241,0.2008,0.008,0.012,0.0964,0.0241,0.4378,0.0843,0.1004,0.0,0.0,0.004,0.008,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Análisis de Nulos

`alcohol`: tomamos el promedio de alcohol por uva para llenar el campo.

`rating_qty`: hay bastantes vinos con muy pocos ratings. En ese caso, podríamos hacer una compleción de nulos con el mínimo.

`tastes + pairing`: hay varios vinos que no tienen sabores ni maridajes. Deberíamos eliminarlos, ya que no son útiles en nuestra app.

`grapes`: podemos simplemente completar todas en 0. Son muy pocos vinos.


In [14]:
pd.DataFrame(wines.isna().sum(), columns=["nulls"]).sort_values("nulls", ascending=False).T

,alcohol,rating_qty,tannis,sweetness,body,style,acidity,mushrooms,pasta,pork,poultry,mature and hard cheese,lamb,lean fish,beef,blue cheese,cured meat,"game (deer, venison)",goat's milk cheese,any junk food will do,appetizers and snacks,aperitif,veal,vegetarian,shellfish,spicy food,"rich fish (salmon, tuna etc)",mild and soft cheese,year,price,Cabernet Franc,Béquignol Noir,Cabernet Sauvignon,Barbera,Albariño,Bonarda,Petit Verdot,Pedro Ximenez,Pais,Mourvedre,Moscatel,Merlot,Mencia,Marsanne,Malvasia,Malbec,Grüner Veltliner,Grenache,Gewürztraminer,Garnacha,Criolla Grande,Chenin Blanc,Chardonnay,Cereza,Shiraz/Syrah,Sémillon,Pinot Gris,Pinot Noir,Riesling,Roussanne,Sangiovese,Sauvignon Blanc,Torrontés,Trousseau,Tannat,Tempranillo,Verdejo,Viognier,spices,tree fruit,rating,name,tropical,yeasty,vegetal,winery,wine_link,image,ageing,black fruit,citrus,dried fruit,earthy,floral,oaky,red fruit,Agrelo,Argentina,Brazil,Cafayate Valley,Calchaqui Valley,Campanha,Famatina,Gualtallary,La Consulta,La Rioja,Las Compuertas,Lujan de Cuyo,Lunlunta,Maipu,Mendoza,Paraje Altamira,Patagonia,Pedernal Valley,Perdriel,Rio Grande do Sul,Rio Negro,Salta,San Carlos,San Juan,San Rafael,Serra Gaúcha,Tulum Valley,Tunuyán,Tupungato,Uco Valley,Vale dos Vinhedos,Vista Flores
nulls,1047,420,363,81,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,28,17,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


_alcohol_

In [ ]:
wines_clean = wines.copy()

grapes = ["Albariño", "Barbera", "Bonarda", "Béquignol Noir", "Cabernet Franc",
          "Cabernet Sauvignon", "Cereza", "Chardonnay", "Chenin Blanc", "Criolla Grande",
          "Garnacha", "Gewürztraminer", "Grenache", "Grüner Veltliner", "Malbec",
          "Malvasia", "Marsanne", "Mencia", "Merlot", "Moscatel", "Mourvedre", "Pais",
          "Pedro Ximenez", "Petit Verdot", "Pinot Gris", "Pinot Noir", "Riesling",
          "Roussanne", "Sangiovese", "Sauvignon Blanc", "Shiraz/Syrah", "Sémillon",
          "Tannat", "Tempranillo", "Torrontés", "Trousseau", "Verdejo", "Viognier"]

# Diccionario con el promedio de alcohol por uva
grape_alcohol_mean = {}

for grape in grapes:
    condition = (wines_clean[grape] == 1) & (wines_clean["alcohol"].notna())
    mean_alcohol = wines_clean.loc[condition, "alcohol"].mean()
    grape_alcohol_mean[grape] = mean_alcohol

grape_alcohol_mean["General"] = wines_clean["alcohol"].mean()

# Imputa el promedio de alcohol por uva a cada vino con alcohol NaN
def impute_alcohol(row):
    if pd.notna(row["alcohol"]):
        return row["alcohol"]
    alcohol_grape_mix = []
    for grape in grapes:
        if row[grape] == 1:
            mean = grape_alcohol_mean.get(grape)
            if not np.isnan(mean):
                alcohol_grape_mix.append(mean)
    # Calcula el promedio de promedios si tiene más de una uva
    if alcohol_grape_mix:
        return round(np.mean(alcohol_grape_mix), 4) 
    else:
        return round(np.mean(grape_alcohol_mean.get("General")), 4)

wines_clean["alcohol"] = wines_clean.apply(impute_alcohol, axis=1)

# Comprobación
pd.DataFrame(wines_clean.isna().sum(), columns=["nulls"]).sort_values("nulls", ascending=False).T

,rating_qty,tannis,sweetness,body,acidity,style,mild and soft cheese,mushrooms,pasta,pork,poultry,mature and hard cheese,lamb,lean fish,beef,blue cheese,cured meat,"game (deer, venison)",goat's milk cheese,any junk food will do,appetizers and snacks,aperitif,veal,vegetarian,shellfish,spicy food,"rich fish (salmon, tuna etc)",year,price,Bonarda,Cabernet Franc,Béquignol Noir,Cabernet Sauvignon,Barbera,Albariño,Pinot Gris,Petit Verdot,Pedro Ximenez,Pais,Mourvedre,Moscatel,Merlot,Mencia,Marsanne,Malvasia,Malbec,Grüner Veltliner,Grenache,Gewürztraminer,Garnacha,Criolla Grande,Chenin Blanc,Chardonnay,Cereza,Shiraz/Syrah,Sémillon,Tannat,Pinot Noir,Riesling,Roussanne,Sangiovese,Sauvignon Blanc,Torrontés,Trousseau,Verdejo,Tempranillo,Viognier,spices,tropical,tree fruit,rating,name,wine_link,yeasty,vegetal,winery,alcohol,image,ageing,black fruit,citrus,dried fruit,earthy,floral,oaky,red fruit,Agrelo,Argentina,Brazil,Cafayate Valley,Calchaqui Valley,Campanha,Famatina,Gualtallary,La Consulta,La Rioja,Las Compuertas,Lujan de Cuyo,Lunlunta,Maipu,Mendoza,Paraje Altamira,Patagonia,Pedernal Valley,Perdriel,Rio Grande do Sul,Rio Negro,Salta,San Carlos,San Juan,San Rafael,Serra Gaúcha,Tulum Valley,Tunuyán,Tupungato,Uco Valley,Vale dos Vinhedos,Vista Flores
nulls,420,363,81,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52,28,17,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Falta imputar los nulos restantes

In [34]:
ut.save_csv(wines_clean, path="../../src/data/transformed/" , filename="wines_clean.csv")

Archivo guardado en: ../../src/data/transformed/wines_clean.csv
